In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv ('Bitcoin_Historical.csv')

df.head

<bound method NDFrame.head of             Date     Price      Open      High       Low     Vol. Change %
0     03/18/2024  67,594.1  68,389.7  68,920.1  66,601.4   78.07K   -1.17%
1     03/17/2024  68,391.2  65,314.2  68,857.7  64,605.5   66.07K    4.71%
2     03/16/2024  65,314.2  69,456.5  70,037.0  64,971.0   75.82K   -5.97%
3     03/15/2024  69,463.7  71,387.1  72,398.1  65,765.6  148.59K   -2.69%
4     03/14/2024  71,387.5  73,066.7  73,740.9  68,717.2  109.43K   -2.30%
...          ...       ...       ...       ...       ...      ...      ...
4988  07/22/2010       0.1       0.1       0.1       0.1    2.16K    0.00%
4989  07/21/2010       0.1       0.1       0.1       0.1    0.58K    0.00%
4990  07/20/2010       0.1       0.1       0.1       0.1    0.26K    0.00%
4991  07/19/2010       0.1       0.1       0.1       0.1    0.57K    0.00%
4992  07/18/2010       0.1       0.0       0.1       0.1    0.08K    0.00%

[4993 rows x 7 columns]>

In [4]:
df = df [['Date', 'Low', 'High', 'Open']]

df

,Date,Low,High,Open
0,03/18/2024,"66,601.4","68,920.1","68,389.7"
1,03/17/2024,"64,605.5","68,857.7","65,314.2"
2,03/16/2024,"64,971.0","70,037.0","69,456.5"
3,03/15/2024,"65,765.6","72,398.1","71,387.1"
4,03/14/2024,"68,717.2","73,740.9","73,066.7"
...,...,...,...,...
4988,07/22/2010,0.1,0.1,0.1
4989,07/21/2010,0.1,0.1,0.1
4990,07/20/2010,0.1,0.1,0.1
4991,07/19/2010,0.1,0.1,0.1


In [5]:
def convertir_columna_a_int(df, columna):
    # Elimina las comas de la columna
    df.loc[:, columna] = df[columna].str.replace(',', '')
    # Convierte la columna a tipo float
    df.loc[:, columna] = df[columna].astype(float)
    # Redondea los valores
    df.loc[:, columna] = df[columna].round()
    # Convierte los valores redondeados a tipo int
    df.loc[:, columna] = df[columna].astype(int)

convertir_columna_a_int(df, 'Low')
convertir_columna_a_int(df, 'High')
convertir_columna_a_int(df, 'Open')


In [6]:
import ta
import pandas as pd

atr_multiple = 1.5
n = 20

df[['avehigh', 'avelow']] = df[['High', 'Low']].rolling(n).mean()
df['avemidprice'] = (df['avehigh'] + df['avelow']) / 2

# get upper and lower bounds to compare to period highs and lows
indicator_atr = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Open'], window=14, fillna=False)
df['atr14'] = indicator_atr.average_true_range()
df['UPB'] = df['avemidprice'] + atr_multiple * df['atr14']
df['LPB'] = df['avemidprice'] - atr_multiple * df['atr14']

# get the period highs and lows
df['rangemaxprice'] = df[['High']].rolling(n).max()
df['rangeminprice'] = df[['Low']].rolling(n).min()
df['sideways'] = 0

def sideways_range(maxp, minp, upb, lpb):
    if maxp < upb and maxp > lpb and minp < upb and minp > lpb:
        return 1
    else:
        return 0

df['sideways'] = df[['rangemaxprice', 'rangeminprice', 'UPB', 'LPB']].apply(lambda x: sideways_range(x['rangemaxprice'], x['rangeminprice'], x['UPB'], x['LPB']), axis=1)

C:\Users\pobla\AppData\Local\Temp\ipykernel_25948\3024210071.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['avehigh', 'avelow']] = df[['High', 'Low']].rolling(n).mean()
C:\Users\pobla\AppData\Local\Temp\ipykernel_25948\3024210071.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['avehigh', 'avelow']] = df[['High', 'Low']].rolling(n).mean()


In [7]:
df

,Date,Low,High,Open,avehigh,avelow,avemidprice,atr14,UPB,LPB,rangemaxprice,rangeminprice,sideways
0,03/18/2024,66601,68920,68389,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0
1,03/17/2024,64605,68857,65314,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0
2,03/16/2024,64971,70037,69456,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0
3,03/15/2024,65765,72398,71387,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0
4,03/14/2024,68717,73740,73066,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4988,07/22/2010,0,0,0,0.0,0.0,0.0,1.187959e-07,1.781939e-07,-1.781939e-07,0.0,0.0,1
4989,07/21/2010,0,0,0,0.0,0.0,0.0,1.103105e-07,1.654657e-07,-1.654657e-07,0.0,0.0,1
4990,07/20/2010,0,0,0,0.0,0.0,0.0,1.024312e-07,1.536467e-07,-1.536467e-07,0.0,0.0,1
4991,07/19/2010,0,0,0,0.0,0.0,0.0,9.511465e-08,1.426720e-07,-1.426720e-07,0.0,0.0,1
